# Caffe2 Introduction

References
* https://caffe2.ai/docs/intro-tutorial.html

In [1]:
from caffe2.python import workspace, model_helper
import numpy as np

## Blobs and Workspace, Tensors

In [2]:
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.88997241 0.12022102]
  [0.88188169 0.18473214]
  [0.71762122 0.11558636]]

 [[0.22579086 0.67422141]
  [0.48671107 0.94477542]
  [0.54582604 0.93478675]]

 [[0.72404967 0.82139218]
  [0.0241157  0.19129789]
  [0.15488574 0.01253563]]

 [[0.36795231 0.81898088]
  [0.92400248 0.66840576]
  [0.31662866 0.38012366]]]
(4, 3, 2)
[[[0.88997241 0.12022102]
  [0.88188169 0.18473214]
  [0.71762122 0.11558636]]

 [[0.22579086 0.67422141]
  [0.48671107 0.94477542]
  [0.54582604 0.93478675]]

 [[0.72404967 0.82139218]
  [0.0241157  0.19129789]
  [0.15488574 0.01253563]]

 [[0.36795231 0.81898088]
  [0.92400248 0.66840576]
  [0.31662866 0.38012366]]]


## Nets and Operators

### 1. Model defintion

In [3]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

True

In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")

In [5]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

In [6]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



### 2. Executing

In [7]:
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net)

True

In [8]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [9]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.09339145 0.10642346 0.10014153 0.10325584 0.09183178 0.09534732
  0.08891211 0.10233209 0.12209567 0.09626896]
 [0.09899452 0.11280565 0.10002844 0.09607407 0.09911832 0.10619459
  0.07705664 0.11145403 0.10433369 0.09394009]
 [0.08286455 0.10791922 0.10609834 0.09013259 0.09178647 0.09616897
  0.08506706 0.11604338 0.12522957 0.09868987]
 [0.09134731 0.11518338 0.09047218 0.08984502 0.09977093 0.10848781
  0.07927989 0.10156581 0.12222464 0.10182317]
 [0.09768469 0.09847265 0.09167515 0.09170575 0.096403   0.09395219
  0.09331473 0.10700324 0.1209162  0.10887244]
 [0.08799846 0.12094094 0.09772131 0.09030902 0.09551021 0.10199878
  0.08268747 0.1148183  0.11389599 0.09411947]
 [0.11328786 0.11041225 0.09950201 0.09428996 0.09200225 0.09499804
  0.08375614 0.10167009 0.11757088 0.09251044]
 [0.08725245 0.11221404 0.10196627 0.09240996 0.09664204 0.1015223
  0.08091603 0.1116172  0.10908668 0.10637297]
 [0.0866763  0.10937949 0.09433612 0.10313403 0.09211618 0.09972405
  0.09426679 

### 3. Backward pass

In [10]:
m.AddGradientOperators([loss])
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net, overwrite=True)

True

In [11]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label

In [12]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [13]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.07764349 0.11906692 0.08852771 0.10767992 0.09314898 0.08110189
  0.1064648  0.09220832 0.09623636 0.13792174]
 [0.09315608 0.1179883  0.09621266 0.11301854 0.08626252 0.07878942
  0.10712076 0.09544351 0.09666031 0.11534771]
 [0.09202261 0.11013588 0.0967541  0.11250325 0.10506929 0.08122766
  0.10346481 0.09218597 0.08264202 0.12399448]
 [0.08944649 0.11206648 0.10214503 0.09155405 0.09619832 0.08243875
  0.10476203 0.09395029 0.09811552 0.129323  ]
 [0.08586247 0.11873516 0.09534384 0.10848465 0.09346242 0.08816012
  0.10321909 0.09627131 0.08954283 0.12091807]
 [0.07998072 0.11492892 0.09025536 0.10086226 0.0940085  0.08645912
  0.11578054 0.08664259 0.09488203 0.1362001 ]
 [0.08601896 0.12322535 0.10309216 0.10297704 0.08651943 0.08301044
  0.08535339 0.0948774  0.09138822 0.14353763]
 [0.08541438 0.11164442 0.10693711 0.10046652 0.08668761 0.09766892
  0.09125784 0.10340477 0.0853864  0.13113196]
 [0.08984394 0.11833447 0.09547494 0.09715635 0.09566005 0.08537985
  0.09069771